In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Cell_Phones_and_Accessories_5.json


<h1 style="background-color:#289489; color:white; font-size:20px; padding:30px;text-align:center;font:serif">Import libraries</h1>

In [2]:
!pip install contractions # Replace with the desired version


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.9 MB/s eta 0:00:00


In [3]:
!pip install gensim
!pip install smart_open

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 42.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tsfresh 0.20.3 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [4]:
!pip install wordcloud
!pip install nltk
!pip install plotly

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from warnings import filterwarnings
filterwarnings("ignore")
import re
import string
import contractions
import string
import keras
from gensim.parsing.preprocessing import STOPWORDS,remove_stopwords
import wordcloud
from tensorflow.keras.utils import to_categorical
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
import plotly.express as px
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Embedding, Input
import plotly.express as px


<h1 style="background-color:#289489; color:white; font-size:20px; padding:30px;text-align:center;font:serif">Load and explore data</h1>

In [6]:
data=pd.read_json('/kaggle/input/Cell_Phones_and_Accessories_5.json',lines=True)

In [7]:
data.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194439 entries, 0 to 194438
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   reviewerID      194439 non-null  object
 1   asin            194439 non-null  object
 2   reviewerName    190920 non-null  object
 3   helpful         194439 non-null  object
 4   reviewText      194439 non-null  object
 5   overall         194439 non-null  int64 
 6   summary         194439 non-null  object
 7   unixReviewTime  194439 non-null  int64 
 8   reviewTime      194439 non-null  object
dtypes: int64(2), object(7)
memory usage: 13.4+ MB


In [9]:
# Removing unnecessary columns
df=data[['reviewText','overall','summary']]

In [10]:
def defineSentiment(rating):
    if rating>=4:
        return 'Positive'
    elif rating==3:
        return 'Neutral'
    else: return 'Negative'

In [11]:
df['sentiment']=df['overall'].apply(defineSentiment)

In [12]:
df['lenOftext']=df['reviewText'].apply(len)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194439 entries, 0 to 194438
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   reviewText  194439 non-null  object
 1   overall     194439 non-null  int64 
 2   summary     194439 non-null  object
 3   sentiment   194439 non-null  object
 4   lenOftext   194439 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 7.4+ MB


In [14]:
pd.set_option('display.float_format',lambda x:'%.2f'%x)
df.describe().T.sort_values(by='std',ascending=False)\
.style.background_gradient(cmap='ocean')\
.bar(subset=['mean',],color='#99a1f0')\
.bar(subset=['max'],color='#86e1e3')

,count,mean,std,min,25%,50%,75%,max
lenOftext,194439.000000,491.839646,749.170243,0.000000,143.000000,248.000000,532.000000,32110.000000
overall,194439.000000,4.129912,1.222499,1.000000,4.000000,5.000000,5.000000,5.000000


While most reviews are relatively short, a few are extremely long. This indicates diverse user behavior in terms of review length.


In [15]:
df[df['lenOftext']==0]

,reviewText,overall,summary,sentiment,lenOftext
548,,4,"Nice Product, If You Do Your Homework",Positive,0
4567,,5,Great all-in-one battery pack with lots of power,Positive,0
4743,,5,Compact Battery with Extras,Positive,0
4810,,5,Good backup power when you need it,Positive,0
5094,,5,Great Cable without the APPLE price,Positive,0
...,...,...,...,...,...
191265,,5,Charges two devices at a time,Positive,0
191352,,5,"Great for kayaking, Galaxy S5",Positive,0
191426,,5,"Excellent, just like their last screen protect...",Positive,0
192042,,5,"Excellent Car Charger, Very Fast!",Positive,0


In [16]:
df_new=df[df['lenOftext']>0]
df_new.reset_index(drop=True,inplace=True)

In [17]:
df_new.head()

,reviewText,overall,summary,sentiment,lenOftext
0,They look good and stick good! I just don't li...,4,Looks Good,Positive,189
1,These stickers work like the review says they ...,5,Really great product.,Positive,160
2,These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,Positive,166
3,Item arrived in great time and was in perfect ...,4,Cute!,Positive,275
4,"awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,Positive,135


In [18]:
pd.set_option('display.float_format',lambda x:'%.2f'%x)
df_new.describe().T.sort_values(by='std',ascending=False)\
.style.background_gradient(cmap='ocean')\
.bar(subset=['mean',],color='#99a1f0')\
.bar(subset=['max'],color='#86e1e3')

,count,mean,std,min,25%,50%,75%,max
lenOftext,194340.000000,492.090198,749.278768,2.000000,143.000000,248.000000,533.000000,32110.000000
overall,194340.000000,4.129881,1.222449,1.000000,4.000000,5.000000,5.000000,5.000000


<div style="background-color:#95b6ed; padding: 30px;">
    <h3 style="color:white; font-size:20px; text-align:center; font-family: serif;">Conclusion</h3>
    <p style="color:black; font-size:16px; text-align:center;">The minimum length of review is 2 and max is 32110, indicating significant variation. Additionally, most of the reviews are positive, highlighting a data imbalance.</p>
</div>


<h1 style="background-color:#289489; color:white; font-size:20px; padding:30px;text-align:center;font:serif">EDA</h1>

In [19]:
val=df_new['sentiment'].value_counts()

In [20]:
px.pie(df_new, names=val.index, values=val.values, title="Distribution of Sentiment",
             color_discrete_sequence=px.colors.sequential.Teal)


In [21]:
rating=df_new['overall'].value_counts()

In [22]:
px.bar(df_new,x=rating.index,y=rating.values,title="Ratings Distribution")

In [23]:
# highest 20 length of reviews and their sentiment
top30=df_new['lenOftext'].nlargest(30).index
df_new_30=df_new.loc[top30]
df_new_30
px.histogram(df_new_30,x='lenOftext',y=df_new_30.index, color='sentiment', title="Top 30 Reviews by Length and Their Sentiments")


<div style="background-color:#95b6ed; padding: 30px;">
    <h3 style="color:white; font-size:20px; text-align:center; font-family: serif;">Conclusion</h3>
    <p style="color:black; font-size:16px; text-align:center;">The abundance of positive reviews contributes to data imbalance, reflecting the tendency of satisfied individuals to share their experiences more extensively.</p>
</div>


<h1 style="background-color:#289489; color:white; font-size:20px; padding:30px;text-align:center;font:serif">Text Preprocessing</h1>

<h3 style="background-color:#a389b0; color:white; font-size:12px; padding:30px;text-align:center;font:serif">Lowercase</h3>

In [24]:
def lower(text):
    return text.lower()
df_new['reviewText']=df_new['reviewText'].apply(lower)
df_new['summary']=df_new['summary'].apply(lower)

<h3 style="background-color:#a389b0; color:white; font-size:12px; padding:30px;text-align:center;font:serif">URL and HTML Tags</h3>

In [25]:
# URL Patterns
def findURL(text):
    expression=r'https://\S+\.\S+\.\S+'
    return bool(re.findall(expression,text))

In [26]:
df_new['url']=df_new['reviewText'].apply(findURL)

In [27]:
df_new['url'].value_counts()

url
False    194331
True          9
Name: count, dtype: int64

In [28]:
# let's remove the URL
def removeURL(text):
    expression=r'https://\S+\.\S+\.\S+'
    return re.sub(expression,'',text)

In [29]:
df_new['reviewText']=df_new['reviewText'].apply(removeURL)

In [30]:
df_new['summary']=df_new['summary'].apply(removeURL)

In [31]:
def findHTML(text):
    expression=r'<[^>]+>'
    return bool(re.findall(expression,text))

In [32]:
df_new['html']=df_new['reviewText'].apply(findHTML)

In [33]:
# No need to remove html tags(none present)
df_new['html'].value_counts()

html
False    194340
Name: count, dtype: int64

<h1 style="background-color:#a389b0; color:white; font-size:12px; padding:30px;text-align:center;font:serif">Fix Contractions</h1>

In [34]:
def fixContractions(text):
    return contractions.fix(text)

In [35]:
df_new['reviewText']=df_new['reviewText'].apply(fixContractions)

In [36]:
df_new['summary']=df_new['summary'].apply(fixContractions)

<h1 style="background-color:#a389b0; color:white; font-size:12px; padding:30px;text-align:center;font:serif">Removing numbers in text</h1>

In [37]:
def removeNumbering(text):
    expression=r'\S+\d+\) '
    return re.sub(expression,'',text)

In [38]:
df_new['reviewText']=df_new['reviewText'].apply(removeNumbering)

In [39]:
df_new['summary']=df_new['summary'].apply(removeNumbering)

<h1 style="background-color:#a389b0; color:white; font-size:12px; padding:30px;text-align:center;font:serif">Remove punctuations without currency percentage sign.Also retain decimal numbers</h1>

In [40]:
def getText(text):
    expressions=r'[^\d+\%|\d+\$|\$\d+|\d+\.\d|\w+\s+]'
    return re.sub(expressions,'',text)

In [41]:
df_new['reviewText']=df_new['reviewText'].apply(getText)

In [42]:
df_new['summary']=df_new['summary'].apply(getText)

In [43]:
def removeExtraWhitespaces(text):
    expression=r'\s+'
    return re.sub(expression," ",text)

In [44]:
df_new['reviewText']=df_new['reviewText'].apply(removeExtraWhitespaces)

In [45]:
df_new['summary']=df_new['summary'].apply(removeExtraWhitespaces)

<h1 style="background-color:#a389b0; color:white; font-size:12px; padding:30px;text-align:center;font:serif">Find emojis and remove if any</h1>

In [46]:
def findemoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return bool(re.findall(emoji_pattern, text))

In [47]:
df_new['isemoji']=df_new['reviewText'].apply(findemoji)

In [48]:
df_new['isemoji'].value_counts()

isemoji
False    194340
Name: count, dtype: int64

<h1 style="background-color:#a389b0; color:white; font-size:12px; padding:30px;text-align:center;font:serif">Remove full stop</h1>

In [49]:
def removeFullStops(text):
    pattern=r'\.\s'
    return re.sub(pattern,' ',text)

In [50]:
df_new['reviewText']=df_new['reviewText'].apply(removeFullStops)

In [51]:
df_new['summary']=df_new['summary'].apply(removeFullStops)

In [52]:
df_new.drop(columns=['url','html','isemoji'],axis=1,inplace=True)

In [53]:
df_new.head()

,reviewText,overall,summary,sentiment,lenOftext
0,they look good and stick good i just do not li...,4,looks good,Positive,189
1,these stickers work like the review says they ...,5,really great product.,Positive,160
2,these are awesome and make my phone look so st...,5,love love love,Positive,166
3,item arrived in great time and was in perfect ...,4,cute,Positive,275
4,awesome stays on and looks great can be used o...,5,leopard home button sticker for iphone 4s,Positive,135


<h1 style="background-color:#a389b0; color:white; font-size:12px; padding:30px;text-align:center;font:serif">Stop word removal</h1>

In [54]:
stopwords=STOPWORDS

In [55]:
def removeStopwords(text):
    return remove_stopwords(text)

In [56]:
df_new['reviewText']=df_new['reviewText'].apply(removeStopwords)

In [57]:
df_new['summary']=df_new['summary'].apply(removeStopwords)

<h1 style="background-color:#a389b0; color:white; font-size:12px; padding:30px;text-align:center;font:serif">lemmatization</h1>

In [58]:
nltk.download('wordnet')
nltk.data.path

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['/root/nltk_data',
 '/usr/share/nltk_data',
 '/usr/local/share/nltk_data',
 '/usr/lib/nltk_data',
 '/usr/local/lib/nltk_data']

In [63]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/


Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [64]:
lemmatizer=WordNetLemmatizer()

In [65]:
def lemmatize(text):
    return ' '.join(lemmatizer.lemmatize(word) for word in text.split())

In [66]:
df_new['reviewText']=df_new['reviewText'].apply(lemmatize)

In [67]:
df_new['summary']=df_new['summary'].apply(lemmatize)

In [68]:
df_new.head()

,reviewText,overall,summary,sentiment,lenOftext
0,look good stick good like rounded shape bumpin...,4,look good,Positive,189
1,sticker work like review say stick great stay ...,5,great product.,Positive,160
2,awesome phone look stylish far year believe ye...,5,love love love,Positive,166
3,item arrived great time perfect condition orde...,4,cute,Positive,275
4,awesome stay look great multiple apple product...,5,leopard home button sticker iphone 4,Positive,135


<h1 style="background-color:#289489; color:white; font-size:20px; padding:30px;text-align:center;font:serif">Tokenization and padding</h1>

In [69]:
corpus=df_new['reviewText'].tolist()

In [70]:
def tokenize(text):
    tokens=Tokenizer()
    tokens.fit_on_texts(text)
    return tokens.texts_to_sequences(text),tokens.word_index

In [71]:
df_new['seq'],vocab_size=tokenize(corpus)

In [72]:
df_new

,reviewText,overall,summary,sentiment,lenOftext,seq
0,look good stick good like rounded shape bumpin...,4,look good,Positive,189,"[17, 9, 300, 9, 3, 1746, 789, 4782, 1806, 579,..."
1,sticker work like review say stick great stay ...,5,great product.,Positive,160,"[826, 8, 3, 54, 390, 300, 4, 182, 1, 257, 761,..."
2,awesome phone look stylish far year believe ye...,5,love love love,Positive,166,"[218, 1, 17, 761, 83, 126, 509, 126, 4, 24]"
3,item arrived great time perfect condition orde...,4,cute,Positive,275,"[95, 316, 4, 13, 67, 820, 152, 30, 4, 207, 217..."
4,awesome stay look great multiple apple product...,5,leopard home button sticker iphone 4,Positive,135,"[218, 182, 17, 4, 426, 121, 15, 235, 106, 1674..."
...,...,...,...,...,...,...
194335,work great like original need extra car house ...,5,work perfect,Positive,118,"[8, 4, 3, 180, 31, 144, 50, 518, 19, 20]"
194336,great product great packaging high quality app...,5,great replacement cable apple certified,Positive,180,"[4, 15, 4, 422, 177, 24, 121, 2253, 8, 4, 9, 2..."
194337,great cable good expensive apple husband daugh...,5,real quality,Positive,758,"[4, 27, 9, 306, 121, 545, 448, 554, 180, 27, 5..."
194338,like becasue work life proof case remove case ...,5,like becasue work life proof ...,Positive,116,"[3, 5698, 8, 110, 1245, 2, 191, 2, 10, 1]"


In [73]:
def maxlen(seq):
    maxlen=0
    for i in range(len(seq)):
        maxlen=max(maxlen,len(seq[i]))
    return maxlen

In [74]:
maxlen=maxlen(df_new['seq'].tolist())

In [75]:
maxlen

2716

In [76]:
sequences=df_new['seq'].tolist()

In [77]:
padded_X=pad_sequences(
    sequences,
    maxlen=maxlen,
    dtype='int32',
    padding='post',
    truncating='post',
    value=0.0
)

In [78]:
padded_X

array([[  17,    9,  300, ...,    0,    0,    0],
       [ 826,    8,    3, ...,    0,    0,    0],
       [ 218,    1,   17, ...,    0,    0,    0],
       ...,
       [   4,   27,    9, ...,    0,    0,    0],
       [   3, 5698,    8, ...,    0,    0,    0],
       [  15,  631, 2080, ...,    0,    0,    0]], dtype=int32)

In [79]:
y=df_new['overall']

In [80]:
y=y-1

In [81]:
y

0         3
1         4
2         4
3         3
4         4
         ..
194335    4
194336    4
194337    4
194338    4
194339    4
Name: overall, Length: 194340, dtype: int64

<h1 style="background-color:#289489; color:white; font-size:20px; padding:30px;text-align:center;font:serif">Splitting</h1>

In [82]:
X_train=padded_X[:136037]

In [83]:
X_test=padded_X[136038:]

In [84]:
len(X_train)

136037

In [85]:
len(X_test)

58302

In [86]:
X_train

array([[   17,     9,   300, ...,     0,     0,     0],
       [  826,     8,     3, ...,     0,     0,     0],
       [  218,     1,    17, ...,     0,     0,     0],
       ...,
       [  309, 10711,  1686, ...,     0,     0,     0],
       [ 1249,    27,     1, ...,     0,     0,     0],
       [   69,    21,   867, ...,     0,     0,     0]], dtype=int32)

In [87]:
y=np.array(y)

In [88]:
y_train=y[:136037]

In [89]:
y_test=y[136038:]

<h1 style="background-color:#289489; color:white; font-size:20px; padding:30px;text-align:center;font:serif">Model Training</h1>

In [90]:
model=Sequential()
model.add(Embedding(input_dim=138203,output_dim=500))
model.add(LSTM(150))
model.add(Dense(5,activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.build(input_shape=(None, 2716))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 2716, 500)      │    69,101,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       390,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           755 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,492,855 (265.09 MB)

 Trainable params: 69,492,855 (265.09 MB)

 Non-trainable params: 0 (0.00 B)

In [91]:
y_train

array([3, 4, 4, ..., 4, 4, 4])

In [92]:
model.fit(X_train,y_train,epochs=10)

Epoch 1/10
4252/4252 ━━━━━━━━━━━━━━━━━━━━ 772s 181ms/step - accuracy: 0.5422 - loss: 1.2746
Epoch 2/10
4252/4252 ━━━━━━━━━━━━━━━━━━━━ 768s 181ms/step - accuracy: 0.5434 - loss: 1.2693
Epoch 3/10
4252/4252 ━━━━━━━━━━━━━━━━━━━━ 768s 181ms/step - accuracy: 0.5421 - loss: 1.2723
Epoch 4/10
4252/4252 ━━━━━━━━━━━━━━━━━━━━ 767s 180ms/step - accuracy: 0.5433 - loss: 1.2689
Epoch 5/10
4252/4252 ━━━━━━━━━━━━━━━━━━━━ 768s 181ms/step - accuracy: 0.5434 - loss: 1.2708
Epoch 6/10
4252/4252 ━━━━━━━━━━━━━━━━━━━━ 768s 181ms/step - accuracy: 0.5434 - loss: 1.2712
Epoch 7/10
4252/4252 ━━━━━━━━━━━━━━━━━━━━ 768s 181ms/step - accuracy: 0.5425 - loss: 1.2723
Epoch 8/10
4252/4252 ━━━━━━━━━━━━━━━━━━━━ 768s 181ms/step - accuracy: 0.5429 - loss: 1.2724
Epoch 9/10
4252/4252 ━━━━━━━━━━━━━━━━━━━━ 769s 181ms/step - accuracy: 0.5453 - loss: 1.2675
Epoch 10/10
4252/4252 ━━━━━━━━━━━━━━━━━━━━ 770s 181ms/step - accuracy: 0.5439 - loss: 1.2701


<h1 style="background-color:#289489; color:white; font-size:20px; padding:30px;text-align:center;font:serif">Model Evaluation</h1>

In [93]:
X_test

array([[1244,  414, 1119, ...,    0,    0,    0],
       [  33, 5639,  137, ...,    0,    0,    0],
       [  75,  241,  258, ...,    0,    0,    0],
       ...,
       [   4,   27,    9, ...,    0,    0,    0],
       [   3, 5698,    8, ...,    0,    0,    0],
       [  15,  631, 2080, ...,    0,    0,    0]], dtype=int32)

In [94]:
pred=model.predict(X_test)

1822/1822 ━━━━━━━━━━━━━━━━━━━━ 129s 71ms/step


The observed bias in model performance can be attributed to the imbalance in the dataset, which can be mitigated using techniques such as undersampling and oversampling.








<div style="background-color:#95b6ed; padding: 30px;">
    <h3 style="color:white; font-size:20px; text-align:center; font-family: serif;">Conclusion</h3>
    <p style="color:black; font-size:16px; text-align:center;">Any suggestions to clarify my concepts and refine my approach are highly appreciated. Your feedback is valuable 😊</p>
</div>

Stay tuned for a more polished version.